In [1]:
#Import Required Packeges
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Read Input Data
df=pd.read_excel(r"C:\Users\Anju\Downloads\Customer Data Set.xlsx")

In [3]:
#Find the shape of the data
df.shape

(128188, 28)

In [4]:
#Find the data types of each variables
df.dtypes

BUSINESSDAYDATE    datetime64[ns]
WORKSTATIONID               int64
TRANSNUMBER                 int64
BEGINTIMESTAMP              int64
ENDTIMESTAMP                int64
OPERATORID                  int64
TRANSCURRENCY              object
PLANT                       int64
TURNOVER                  float64
TRANSCOUNTER                int64
RETAILTYPECODE              int64
ITEMID                      int64
RETAILQUANTITY            float64
SALESUOM                   object
SALESAMOUNT               float64
NORMALSALESAMT            float64
COST                      float64
PROMOTIONID               float64
ENTRYMETHODCODE            object
ACTUALUNITPRICE           float64
MATERIALNUMBER              int64
MERCHANDISECAT              int64
BASEQUANTITY              float64
BASEUOM                    object
TAXINC                    float64
ITEMDISC                  float64
DISTVOID                   object
ZZCUSTOMER                float64
dtype: object

In [5]:
# find if there is any duplicate records
df.duplicated().sum()

1068

In [6]:
#Drop Duplicates
df=df.drop_duplicates()

In [7]:
import datetime as dt
df['BEGINTIMESTAMP']=df['BEGINTIMESTAMP'].astype(str)
df['BEGINTIMESTAMP']=pd.to_datetime(df['BEGINTIMESTAMP'],format='%Y%m%d%H%M%S')

In [8]:
df['ENDTIMESTAMP']=df['ENDTIMESTAMP'].astype(str)
df['ENDTIMESTAMP']=pd.to_datetime(df['ENDTIMESTAMP'],format='%Y%m%d%H%M%S')

In [9]:
df['BEGINDATE']=pd.to_datetime(df['BEGINTIMESTAMP'].dt.strftime('%Y/%m/%d'))
df['ENDDATE']=pd.to_datetime(df['ENDTIMESTAMP'].dt.strftime('%Y/%m/%d'))

In [10]:
a=max(df['ENDDATE'])

In [11]:
distvoid=df['DISTVOID']
x = distvoid.str.contains('X', regex=True)
x.fillna(0, inplace=True)
x = x.astype(int)
df['item_canceled'] = x
df.head()

,BUSINESSDAYDATE,WORKSTATIONID,TRANSNUMBER,BEGINTIMESTAMP,ENDTIMESTAMP,OPERATORID,TRANSCURRENCY,PLANT,TURNOVER,TRANSCOUNTER,...,MERCHANDISECAT,BASEQUANTITY,BASEUOM,TAXINC,ITEMDISC,DISTVOID,ZZCUSTOMER,BEGINDATE,ENDDATE,item_canceled
0,2022-01-01,2,1161918,2022-01-01 17:52:14,2022-01-01 17:52:51,153328,AED,2520,49.69,1,...,4001001,1.000,EA,0.43,33.52,NaN,1.784520e+15,2022-01-01,2022-01-01,0
1,2022-01-01,2,1161918,2022-01-01 17:52:14,2022-01-01 17:52:51,153328,AED,2520,49.69,1,...,6004002,2.000,EA,0.10,7.23,NaN,1.784520e+15,2022-01-01,2022-01-01,0
2,2022-01-01,2,1161918,2022-01-01 17:52:14,2022-01-01 17:52:51,153328,AED,2520,49.69,1,...,6001003,0.251,KG,0.12,8.54,NaN,1.784520e+15,2022-01-01,2022-01-01,0
3,2022-01-01,2,1161918,2022-01-01 17:52:14,2022-01-01 17:52:51,153328,AED,2520,49.69,1,...,7005007,2.000,EA,0.15,11.37,NaN,1.784520e+15,2022-01-01,2022-01-01,0
4,2022-01-01,2,1161918,2022-01-01 17:52:14,2022-01-01 17:52:51,153328,AED,2520,49.69,1,...,7004001,1.000,EA,0.10,7.49,NaN,1.784520e+15,2022-01-01,2022-01-01,0


In [12]:
df = df.loc[df['item_canceled'] == 0,:]
df.reset_index(drop=True,inplace=True)

In [13]:
df['ZZCUSTOMER']=df['ZZCUSTOMER'].astype(str)
df = df[df['ZZCUSTOMER'].notna()]
df.reset_index(drop=True,inplace=True)

In [14]:
NewMatrix=df.groupby(['ZZCUSTOMER'],as_index=False).agg({'BEGINDATE': lambda x: (a - x.max()).days,'TRANSNUMBER':'count','SALESAMOUNT':'sum'}).rename(columns={'BEGINDATE':'Recency','TRANSNUMBER':'Frequency','SALESAMOUNT':'MonetaryValue'})

In [15]:
NewMatrix=NewMatrix.loc[NewMatrix['MonetaryValue'] >0,:]
NewMatrix.reset_index(drop=True,inplace=True)

In [16]:
NewMatrix.drop(NewMatrix.loc[NewMatrix.ZZCUSTOMER =='nan'].index,inplace=True)

In [17]:
r_labels = range(5, 1, -1)
r_quartiles = pd.qcut(NewMatrix['Recency'], 4, labels = r_labels)
NewMatrix = NewMatrix.assign(R = r_quartiles.values)

In [18]:
f_labels = range(1,5,1)
m_labels = range(1,5,1)
f_quartiles = pd.qcut(NewMatrix['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(NewMatrix['MonetaryValue'], 4, labels = m_labels)
NewMatrix = NewMatrix.assign(F = f_quartiles.values)
NewMatrix = NewMatrix.assign(M = m_quartiles.values)

In [19]:
def join_rfm(x):
    return str(x['R']) + str(x['F']) + str(x['M'])

NewMatrix['RFM Score'] = NewMatrix.apply(join_rfm, axis=1)

In [20]:
NewMatrix['RFM Score']=NewMatrix['RFM Score'].astype(str)

In [21]:
NewMatrix

,ZZCUSTOMER,Recency,Frequency,MonetaryValue,R,F,M,RFM Score
0,1784200000000117.0,1,10,145.80,5,2,3,523
1,1784200002408014.0,2,2,135.00,4,1,3,413
2,1784200005114623.0,5,3,10.85,2,1,1,211
3,1784200007056575.0,2,1,109.00,4,1,3,413
4,1784200007338669.0,5,3,91.45,2,1,2,212
5,1784520000100050.0,1,24,154.75,5,4,4,544
6,1784520000100065.0,5,4,45.60,2,2,1,221
7,1784520000100080.0,3,6,15.90,2,2,1,221
8,1784520000100096.0,1,13,88.80,5,3,2,532
9,1784520000100110.0,2,2,14.85,4,1,1,411
